In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2020mlfinalexamp5/submission_sample.csv
/kaggle/input/2020mlfinalexamp5/train.csv
/kaggle/input/2020mlfinalexamp5/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/2020mlfinalexamp5/train.csv')
test = pd.read_csv('/kaggle/input/2020mlfinalexamp5/test.csv')
submit = pd.read_csv('/kaggle/input/2020mlfinalexamp5/submission_sample.csv')

In [3]:
train.drop(['inst_id','sido','ownerChange'],axis=1,inplace=True)
test.drop(['inst_id','sido','ownerChange'],axis=1,inplace=True)

In [4]:
open = {'open':1,' close':0}

train['OC'] = train['OC'].map(open)

In [5]:
inst = {'nursing_hospital': 0, 'hospital': 1, 'general_hospital': 2,
        'clinic': 3, 'traditional_hospital': 4, 'traditional_clinic': 5,
        'dental_clinic': 6}

train['instkind'] = train['instkind'].map(inst)
test['instkind'] = test['instkind'].map(inst)

In [6]:
from sklearn.impute import SimpleImputer
im = SimpleImputer(missing_values=np.nan, strategy='mean')

test = im.fit_transform(test)
train = im.fit_transform(train)

test = pd.DataFrame(test)
train = pd.DataFrame(train)

x_train = train.loc[:,1:54]
y_train = train.loc[:,0]

test = test.loc[:,0:53]

In [7]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()

x_train, y_train = sm.fit_resample(x_train,y_train)

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

std = StandardScaler()

In [9]:
param_grid = {'kernel': ('rbf','linear','poly'),'C':[0.1,1,10,100],'gamma':[0.0001,0.0005,0.001,0.1]}

cv = GridSearchCV(SVC(), param_grid, cv = 5)

clf = make_pipeline(std,cv)
clf.fit(x_train,y_train)
pred = clf.predict(test)

In [10]:
submit['OC'] = pred.astype(int)

submit.to_csv('submission.csv',index=None, header=True)